In [1]:
import textdistance
import json
from preprocess import preprocess_c

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1129)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1129)>
[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1129)>
[nltk_data] Error loading omw-1.4: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1129)>


In [2]:
outputfile = 'data/results/syntactic/results-syn.csv'
hammingfile = 'data/results/syntactic/results-syn-hamming.csv'
jaccardfile = 'data/results/syntactic/results-syn-jaccard.csv'
levenfile = 'data/results/syntactic/results-syn-leven.csv'
sorensenfile = 'data/results/syntactic/results-syn-sorensen.csv'
jarofile = 'data/results/syntactic/results-syn-jaro.csv'
tverskyfile = 'data/results/syntactic/results-syn-tversky.csv'
overlapfile = 'data/results/syntactic/results-syn-overlap.csv'
bagfile = 'data/results/syntactic/results-syn-bag.csv'
needlefile = 'data/results/syntactic/results-syn-needle.csv'
lcsstrfile = 'data/results/syntactic/results-syn-lcsstr.csv'

w1 = 'data/raw/ms-aladin-witness1-plagiarism.txt' 
w2 = 'data/raw/ms-aladin-witness2-plagiarism.txt'
w3 = 'data/raw/ts-aladin-witness1-plagiarism.txt'
w4 = 'data/raw/ts-aladin-witness2-plagiarism.txt'

In [3]:
with open('data/stopwords-iso.json', encoding='utf-8') as f:
    stopwords = json.load(f)['nl']
    
other_stopwords = ['zooals','zoo','den','ter','ten','k','uwe','laten','gaat', 't']
stopwords.extend(other_stopwords)
    
with open(w1, 'r') as f:
    w1_raw = f.read()
    
with open(w2, 'r') as f:
    w2_raw = f.read()

with open(w3, 'r') as f:
    w3_raw = f.read()
    
with open(w4, 'r') as f:
    w4_raw = f.read()

w1_preprocessed = preprocess_c(w1_raw, [])
w2_preprocessed = preprocess_c(w2_raw, [])
w3_preprocessed = preprocess_c(w3_raw, [])
w4_preprocessed = preprocess_c(w4_raw, [])

In [4]:
from csv import writer

def append_list_as_row(file_name, list_of_elem):
    with open(file_name, 'a+', newline='') as write_obj:
        csv_writer = writer(write_obj)
        csv_writer.writerow(list_of_elem)
        
def remove_stopwords(source, sw):
    if isinstance(source, str):
        return preprocess_c(source, sw)
    if isinstance(source, list):
        return [w for w in source if not w in sw]
    return ''

In [5]:
header_row = ['source_token', 'source', 'target', 'context_size', 'source_context_words', 'source_context_words_nstopw', 'source_context_inline', 'source_context_inline_nstopw', 'source_context_span', 'target_context_words', 'target_context_words_nstopw', 'target_context_inline', 'target_context_inline_nstopw', 'target_context_span', 'similarity_metric', 'similarity_score']
append_list_as_row(outputfile, header_row)
append_list_as_row(hammingfile, header_row)
append_list_as_row(levenfile, header_row)
append_list_as_row(jarofile, header_row)
append_list_as_row(jaccardfile, header_row)
append_list_as_row(lcsstrfile, header_row)
append_list_as_row(tverskyfile, header_row)
append_list_as_row(overlapfile, header_row)
append_list_as_row(bagfile, header_row)
append_list_as_row(needlefile, header_row)
append_list_as_row(sorensenfile, header_row)


In [6]:
def find_indices(list_to_check, item_to_find):
    indices = []
    for idx, value in enumerate(list_to_check):
        if value == item_to_find:
            indices.append(idx)
    return indices

def find_context(search_token, list_of_tokens, context_size=1):
    result = []
    matching_indices = find_indices(list_of_tokens, search_token)
    for matching_index in matching_indices:
        curr_match = {}
        curr_match['index'] = matching_index
        curr_match['span'] = [matching_index, len(search_token)]
        if ((matching_index-context_size >= 0) and (matching_index+context_size+1 <= len(list_of_tokens)-1)):
            context = list_of_tokens[matching_index-context_size: matching_index+context_size+1]
        elif ((matching_index-context_size < 0) and (matching_index+context_size+1 <= len(list_of_tokens)-1)):
            context = list_of_tokens[: matching_index+context_size+1]
        elif ((matching_index-context_size >= 0) and (matching_index+context_size+1 > len(list_of_tokens)-1)):
            context = list_of_tokens[matching_index-context_size:]
        else: # both (not possible)
            return []
        
        inline = ' '.join(context)
        curr_match['inline'] = inline
        ind = context.index(search_token)
        context.pop(ind)
        words = context
        curr_match['words'] = words
        result.append(curr_match)
        
    return result

In [7]:
# print(preprocessed_sentences)
# print()
tokens_already_processed = []
import needle

def analyse_token_context(source, targets, context_size_range=[1,5]):
    global tokens_already_processed
    
    results = []
    for token in source['data']:
        # print(token)
        if ((token not in tokens_already_processed) and (token not in stopwords)): # do not process a token you already have, do not process a stopword token either
            tokens_already_processed.append(token)
            for i in range(context_size_range[0], context_size_range[1]+1):
                source_contexts = find_context(token, list_of_tokens=source['data'], context_size=i)
                # print(len(source_contexts))
                for source_context in source_contexts:
                    for target in targets:
                        target_contexts = find_context(token, list_of_tokens=target['data'], context_size=i)
                        
                        if ((len(target_contexts) >= 1) and len(target_contexts[0]['inline']) > 0):
                            for target_context in target_contexts:
                                
                                # Edit-distance based measures use inline text comparison
                                metric = 'hamming'
                                score = textdistance.hamming.normalized_similarity(remove_stopwords(source_context['inline'], stopwords), remove_stopwords(target_context['inline'], stopwords)) # hamming 
                                results.append([token, source['name'], target['name'], i, source_context['words'], remove_stopwords(source_context['words'], stopwords), source_context['inline'], remove_stopwords(source_context['inline'], stopwords), source_context['span'], target_context['words'], remove_stopwords(target_context['words'], stopwords), target_context['inline'], remove_stopwords(target_context['inline'], stopwords), target_context['span'], metric, score])      
                                metric = 'levenshtein'
                                score = textdistance.levenshtein.normalized_similarity(remove_stopwords(source_context['inline'], stopwords), remove_stopwords(target_context['inline'], stopwords)) # levenshtein 
                                results.append([token, source['name'], target['name'], i, source_context['words'], remove_stopwords(source_context['words'], stopwords), source_context['inline'], remove_stopwords(source_context['inline'], stopwords), source_context['span'], target_context['words'], remove_stopwords(target_context['words'], stopwords), target_context['inline'], remove_stopwords(target_context['inline'], stopwords), target_context['span'], metric, score])      
                                metric = 'jaro_winkler'
                                score = textdistance.jaro_winkler.normalized_similarity(remove_stopwords(source_context['inline'], stopwords), remove_stopwords(target_context['inline'], stopwords)) # jaro_winkler
                                results.append([token, source['name'], target['name'], i, source_context['words'], remove_stopwords(source_context['words'], stopwords), source_context['inline'], remove_stopwords(source_context['inline'], stopwords), source_context['span'], target_context['words'], remove_stopwords(target_context['words'], stopwords), target_context['inline'], remove_stopwords(target_context['inline'], stopwords), target_context['span'], metric, score])      
                                metric = 'needleman_wunsch'
                                needle_alignment = needle.NeedlemanWunsch(remove_stopwords(source_context['inline'], stopwords), remove_stopwords(target_context['inline'], stopwords))
                                # needle_alignment.change_matrix(core.ScoreMatrix(1, -1, -1))
                                needle_alignment.align()
                                score = needle_alignment.get_score()
                                results.append([token, source['name'], target['name'], i, source_context['words'], remove_stopwords(source_context['words'], stopwords), source_context['inline'], remove_stopwords(source_context['inline'], stopwords), source_context['span'], target_context['words'], remove_stopwords(target_context['words'], stopwords), target_context['inline'], remove_stopwords(target_context['inline'], stopwords), target_context['span'], metric, score])      

                                # Token-based measures use words comparison
                                metric = 'jaccard'
                                score = textdistance.jaccard.normalized_similarity(remove_stopwords(source_context['words'], stopwords), remove_stopwords(target_context['words'], stopwords)) # jaccard
                                results.append([token, source['name'], target['name'], i, source_context['words'], remove_stopwords(source_context['words'], stopwords), source_context['inline'], remove_stopwords(source_context['inline'], stopwords), source_context['span'], target_context['words'], remove_stopwords(target_context['words'], stopwords), target_context['inline'], remove_stopwords(target_context['inline'], stopwords), target_context['span'], metric, score])      
                                metric = 'sorensen'
                                score = textdistance.sorensen.normalized_similarity(remove_stopwords(source_context['words'], stopwords), remove_stopwords(target_context['words'], stopwords)) # sorensen 
                                results.append([token, source['name'], target['name'], i, source_context['words'], remove_stopwords(source_context['words'], stopwords), source_context['inline'], remove_stopwords(source_context['inline'], stopwords), source_context['span'], target_context['words'], remove_stopwords(target_context['words'], stopwords), target_context['inline'], remove_stopwords(target_context['inline'], stopwords), target_context['span'], metric, score])      
                                metric = 'tversky'
                                score = textdistance.tversky.normalized_similarity(remove_stopwords(source_context['words'], stopwords), remove_stopwords(target_context['words'], stopwords)) # tversky 
                                results.append([token, source['name'], target['name'], i, source_context['words'], remove_stopwords(source_context['words'], stopwords), source_context['inline'], remove_stopwords(source_context['inline'], stopwords), source_context['span'], target_context['words'], remove_stopwords(target_context['words'], stopwords), target_context['inline'], remove_stopwords(target_context['inline'], stopwords), target_context['span'], metric, score])        
                                metric = 'overlap'
                                score = textdistance.overlap.normalized_similarity(remove_stopwords(source_context['words'], stopwords), remove_stopwords(target_context['words'], stopwords)) # overlap
                                results.append([token, source['name'], target['name'], i, source_context['words'], remove_stopwords(source_context['words'], stopwords), source_context['inline'], remove_stopwords(source_context['inline'], stopwords), source_context['span'], target_context['words'], remove_stopwords(target_context['words'], stopwords), target_context['inline'], remove_stopwords(target_context['inline'], stopwords), target_context['span'], metric, score])      
                                metric = 'bag'
                                score = textdistance.bag.normalized_similarity(remove_stopwords(source_context['words'], stopwords), remove_stopwords(target_context['words'], stopwords)) # bag
                                results.append([token, source['name'], target['name'], i, source_context['words'], remove_stopwords(source_context['words'], stopwords), source_context['inline'], remove_stopwords(source_context['inline'], stopwords), source_context['span'], target_context['words'], remove_stopwords(target_context['words'], stopwords), target_context['inline'], remove_stopwords(target_context['inline'], stopwords), target_context['span'], metric, score])      

                                # Sequence-based measures
                                # metric = 'lcsseq'
                                # score = textdistance.lcsseq.normalized_similarity(source_context['inline'], target_context['inline']) # lcsseq
                                # results.append([token, source['name'], target['name'], i, source_context['words'], source_context['inline'], target_context['words'], target_context['inline'], metric, score])      
                                metric = 'lcsstr'
                                score = textdistance.lcsstr.normalized_similarity(remove_stopwords(source_context['inline'], stopwords), remove_stopwords(target_context['inline'], stopwords)) # lcsstr 
                                results.append([token, source['name'], target['name'], i, source_context['words'], remove_stopwords(source_context['words'], stopwords), source_context['inline'], remove_stopwords(source_context['inline'], stopwords), source_context['span'], target_context['words'], remove_stopwords(target_context['words'], stopwords), target_context['inline'], remove_stopwords(target_context['inline'], stopwords), target_context['span'], metric, score])                    
    return results


In [8]:
def write_results_to_file(outputfile, results):
    for item in results:
        append_list_as_row(outputfile, item)

In [9]:
w1_results = analyse_token_context(source={'data' : w1_preprocessed, 'name' : 'w1'},targets=[{'data' : w2_preprocessed, 'name' : 'w2'},{'data' : w3_preprocessed, 'name' : 'w3'},{'data' : w4_preprocessed, 'name' : 'w4'}], context_size_range=[1,10])
# write_results_to_file(outputfile, w1_results)
print('source: w1 - DONE')
# w2_results = analyse_token_context(source={'data' : w2_preprocessed, 'name' : 'w2'},targets=[{'data' : w1_preprocessed, 'name' : 'w1'},{'data' : w3_preprocessed, 'name' : 'w3'},{'data' : w4_preprocessed, 'name' : 'w4'}], context_size_range=[1,10])
# write_results_to_file(outputfile, w2_results)
# print('source: w2 - DONE')
# w3_results = analyse_token_context(source={'data' : w3_preprocessed, 'name' : 'w3'},targets=[{'data' : w1_preprocessed, 'name' : 'w1'},{'data' : w2_preprocessed, 'name' : 'w2'},{'data' : w4_preprocessed, 'name' : 'w4'}], context_size_range=[1,10])
# write_results_to_file(outputfile, w3_results)
# print('source: w3 - DONE')
# w4_results = analyse_token_context(source={'data' : w4_preprocessed, 'name' : 'w4'},targets=[{'data' : w1_preprocessed, 'name' : 'w1'},{'data' : w2_preprocessed, 'name' : 'w2'},{'data' : w3_preprocessed, 'name' : 'w3'}], context_size_range=[1,10])
# write_results_to_file(outputfile, w4_results)
# print('source: w4 - DONE')
print('FINISHED!')

source: w1 - DONE
FINISHED!


In [10]:
len(w1_results)

811400

In [11]:
for item in w1_results:
    if item[len(item)-2] == 'hamming':
        write_results_to_file(hammingfile, [item])
    if item[len(item)-2] == 'sorensen':
        write_results_to_file(sorensenfile, [item])
    if item[len(item)-2] == 'jaccard':
        write_results_to_file(jaccardfile, [item])
    if item[len(item)-2] == 'levenshtein':
        write_results_to_file(levenfile, [item])
    if item[len(item)-2] == 'jaro_winkler':
        write_results_to_file(jarofile, [item])
    if item[len(item)-2] == 'needleman_wunsch':
        write_results_to_file(needlefile, [item])
    if item[len(item)-2] == 'tversky':
        write_results_to_file(tverskyfile, [item])
    if item[len(item)-2] == 'overlap':
        write_results_to_file(overlapfile, [item])
    if item[len(item)-2] == 'bag':
        write_results_to_file(bagfile, [item])
    if item[len(item)-2] == 'lcsstr':
        write_results_to_file(lcsstrfile, [item])

In [ ]:
print(len(list(set(tokens_already_processed))))

In [ ]:
len(w1_results)

In [ ]:
# print(textdistance.hamming.normalized_similarity('test', 'text')) # hamming 